# Vision Encoder model

> CNN for image feature extraction.

In [ ]:
#| default_exp models.vision

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 
import numpy as np
import torch
import torch.distributions as td
import torch.nn as nn
from fastcore.utils import *

from torch import nn
from torch.nn import functional as F

In [ ]:
#| export
ENCODER_LAYERS_CONFIG = {
    # L1
    "a": [(2, 32, 5, 1, 0), (32, 32, 4, 2, 0), (32, 32, 3, 1, 0), (32, 16, 1, 1, 0)],
    "b": [(2, 16, 5, 1, 0), (16, 32, 4, 2, 0), (32, 32, 3, 1, 0), (32, 16, 1, 1, 0)],
    "c": [(2, 16, 5, 1, 0), (16, 16, 4, 2, 0), (16, 16, 3, 1, 0)],
    "f": [(2, 16, 5, 1, 0), (16, 16, 5, 2, 0), (16, 16, 5, 1, 2)],
    "g": [(2, 32, 5, 1, 0), (32, 32, 5, 2, 0), (32, 32, 5, 1, 2), (32, 16, 1, 1, 0)],
    "h": [(2, 16, 5, 1, 0), (16, 16, 5, 2, 0), (16, 16, 3, 1, 0)],
    "i": [(2, 16, 5, 1, 0), (16, 16, 5, 2, 0), (16, 16, 3, 1, 1)],
    "i_fc": [
        (2, 16, 5, 1, 0),
        (16, 16, 5, 2, 0),
        (16, 16, 3, 1, 1),
        ("fc", 13456, 512),
    ],
    "i_b": [(6, 16, 5, 1, 0), (16, 16, 5, 2, 0), (16, 16, 3, 1, 0)],
    "d4rl_a": [
        (6, 16, 5, 1, 0),
        (16, 32, 5, 2, 0),
        (32, 32, 3, 1, 0),
        (32, 32, 3, 1, 1),
        (32, 16, 1, 1, 0),
    ],
    "d4rl_b": [
        (6, 16, 5, 1, 0),
        (16, 32, 5, 2, 0),
        (32, 32, 3, 1, 0),
        (32, 32, 3, 1, 1),
        (32, 32, 3, 1, 1),
        (32, 16, 1, 1, 0),
    ],
    "d4rl_c": [
        (6, 16, 5, 1, 0),
        (16, 32, 5, 2, 0),
        (32, 32, 3, 1, 0),
        (32, 32, 3, 1, 1),
        (32, 32, 3, 1, 1),
    ],
    "j": [(2, 32, 5, 1, 0), (32, 32, 5, 2, 0), (32, 32, 3, 1, 1), (32, 16, 1, 1, 0)],
    "k": [(2, 16, 5, 1, 0), (16, 32, 5, 2, 0), (32, 32, 3, 1, 1), (32, 16, 1, 1, 0)],
    # L2
    "d": [(16, 16, 3, 1, 0), (16, 16, 3, 1, 0)],
    "e": [
        ("pad", (0, 1, 0, 1)),
        (16, 16, 3, 1, 0),
        ("avg_pool", 2, 2, 0),
        (16, 16, 3, 1, 0),
    ],
    "l2a": [(16, 16, 5, 1, 2), (16, 16, 5, 2, 2), (16, 16, 3, 1, 1)],  # (8, 16, 15, 15)
    "l2b": [(16, 16, 3, 1, 1), (16, 16, 3, 2, 1), (16, 16, 3, 1, 1)],  # (8, 16, 15, 15)
    "l2c": [(16, 32, 5, 1, 2), (32, 32, 5, 2, 2), (32, 32, 3, 1, 1)],  # (8, 32, 15, 15)
    "l2d": [(16, 32, 3, 1, 1), (32, 32, 3, 2, 1), (32, 32, 3, 1, 1)],  # (8, 32, 15, 15)
    "l2e": [(16, 16, 3, 2, 1), (16, 16, 3, 1, 1)],
}

In [ ]:
#| hide
ENCODER_LAYERS_CONFIG['d4rl_a']

[(6, 16, 5, 1, 0),
 (16, 32, 5, 2, 0),
 (32, 32, 3, 1, 0),
 (32, 32, 3, 1, 1),
 (32, 16, 1, 1, 0)]

### Other Encoder

In [ ]:
#| export
class PassThrough(nn.Module):
    def forward(self, x):
        return x


In [ ]:
# #| export
# from torch import nn

# class SequenceBackbone(nn.Module):
#     def __init__(self):
#         """
#         collapse T and BS dimensions prior to passing to backbone
#         afterwards reshape to original shape
#         """
#         super().__init__()
#         self.output_position_dim = 0

#     def _remove_pos_component_for_spatial(self, embeddings):
#         """
#         remove the position component from spatial embeddings

#         Input:
#             embeddings: tensor
#             (T, BS, Ch, W, H) or
#             (BS, Ch, W, H) or
#             (T, BS, H) or
#             (BS, H)
#         """
#         og_shape = tuple(embeddings.shape)
#         flattened_input = len(og_shape) < 4

#         # first reshape to spatial dimension if needed
#         if flattened_input:
#             spatial_shape = (*embeddings.shape[:-1], *self.output_dim)
#             embeddings = embeddings.view(spatial_shape)

#         position_channels = self.output_position_dim[0]

#         # remove the position dimensions
#         if len(embeddings.shape) == 5:
#             embeddings = embeddings[:, :, :-position_channels]
#         elif len(embeddings.shape) == 4:
#             embeddings = embeddings[:, :-position_channels]

#         # reflatten tensor if needed
#         if flattened_input:
#             embeddings = embeddings.view(*og_shape[:-1], -1)

#         return embeddings

#     def remove_pos_component(self, embeddings):
#         """
#         remove the position component from embeddings
#         Input:
#             embeddings: tensor
#             (T, BS, Ch, W, H) or
#             (BS, Ch, W, H) or
#             (T, BS, H) or
#             (BS, H)
#         """
#         if not self.output_position_dim:
#             return embeddings

#         if isinstance(self.output_dim, int):
#             return embeddings[..., : -self.output_position_dim]
#         else:
#             return self._remove_pos_component_for_spatial(embeddings)

#     def forward_multiple(self, x, position=None):
#         """
#         input:
#             x: [T, BS, *] or [BS, T, *]
#         output:
#             x: [T, BS, *] or [T, BS, *]
#         """

#         # if no time dimension, just feed it directly to backbone
#         if x.dim() == 2 or x.dim() == 4:
#             if position is not None:
#                 output = self.forward(x, position)
#             else:
#                 output = self.forward(x)
#             return output

#         state = x.flatten(0, 1)
#         if position is not None:
#             position = position.flatten(0, 1)
#             output = self.forward(state, position)
            
#         else:
#             output = self.forward(state)

#         state = output.encodings
#         new_shape = x.shape[:2] + state.shape[1:]
#         state = state.reshape(new_shape)

        
#         return state

In [ ]:
# #| export
# from mawm.models.vision.enums import BackboneOutput
# class MLPNet(nn.Module):
#     def __init__(self, output_dim: int = 64):
#         super().__init__()
#         self.fc1 = nn.Linear(28 * 28, output_dim)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(output_dim, output_dim)

#     def forward(self, x):
#         out = x.flatten(1)
#         out = self.fc1(out)
#         out = self.relu(out)
#         out = self.fc2(out)
#         out = BackboneOutput(encodings=out)
#         return out

In [ ]:
# #| export
# from mawm.models.vision.enums import BackboneOutput
# from mawm.models.vision.base import SequenceBackbone
# import torch
# import torch.nn as nn

# class MeNet5(SequenceBackbone):
#     def __init__(
#         self,
#         input_dim: int,
#         output_dim: int = 64,
#         input_channels: int = 1,
#         width_factor: int = 1,
#         conv_out_dim: int = 9 * 32,
#         backbone_norm: str = "batch_norm",
#         backbone_pool: str = "backbone_pool",
#         backbone_final_fc: bool = True,
#     ):
#         super().__init__()
#         self.width_factor = width_factor
#         self.conv_out_dim = conv_out_dim
#         self.backbone_final_fc = backbone_final_fc
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(
#                 input_channels, 16 * width_factor, kernel_size=5, stride=2, padding=2
#             ),
#             nn.ReLU(),
#             (
#                 nn.BatchNorm2d(16 * width_factor)
#                 if backbone_norm == "batch_norm"
#                 else nn.GroupNorm(4, 16 * width_factor)
#             ),
#             nn.Conv2d(
#                 16 * width_factor, 32 * width_factor, kernel_size=5, stride=2, padding=2
#             ),
#             nn.ReLU(),
#             (
#                 nn.BatchNorm2d(32 * width_factor)
#                 if backbone_norm == "batch_norm"
#                 else nn.GroupNorm(4, 32 * width_factor)
#             ),
#             nn.Conv2d(
#                 32 * width_factor, 32 * width_factor, kernel_size=3, stride=1, padding=1
#             ),
#             nn.ReLU(),
#             (
#                 nn.BatchNorm2d(32 * width_factor)
#                 if backbone_norm == "batch_norm"
#                 else nn.GroupNorm(4, 32 * width_factor)
#             ),
#         )
#         if backbone_pool == "avg_pool":
#             self.pool = nn.AvgPool2d(2, stride=2)
#         else:
#             self.pool = nn.Conv2d(
#                 in_channels=32 * width_factor, out_channels=32, kernel_size=1
#             )
#         sample_input = torch.randn(input_dim).unsqueeze(0)
#         sample_output = self.pool(self.layer1(sample_input)).reshape(1, -1)
#         if backbone_final_fc:
#             self.fc = nn.Linear(sample_output.shape[-1], output_dim)

#     def forward(self, x):
#         out = self.layer1(x)  # [bs,64,16,16]
#         out = self.pool(out)  # [bs, 32, 16, 16]
#         if self.backbone_final_fc:
#             out = out.reshape(out.size(0), -1)
#             out = self.fc(out)
#         out = BackboneOutput(encodings=out)
#         return out


In [ ]:
# #| export
# class ResizeConv2d(nn.Module):
#     def __init__(
#         self,
#         in_channels,
#         out_channels,
#         kernel_size,
#         scale_factor,
#         mode="nearest",
#         groups=1,
#         bias=False,
#         padding=1,
#     ):
#         super().__init__()
#         self.scale_factor = scale_factor
#         self.mode = mode
#         self.conv = nn.Conv2d(
#             in_channels,
#             out_channels,
#             kernel_size,
#             stride=1,
#             padding=padding,
#             groups=groups,
#             bias=bias,
#         )

#     def forward(self, x):
#         x = F.interpolate(x, scale_factor=self.scale_factor, mode=self.mode)
#         x = self.conv(x)
#         x = BackboneOutput(encodings=x)
#         return x


In [ ]:
# #| export

# class Canonical(nn.Module):
#     def __init__(self, output_dim: int = 64):
#         super().__init__()
#         res = int(np.sqrt(output_dim / 64))
#         assert (
#             res * res * 64 == output_dim
#         ), "canonical backbone resolution error: cant fit desired output_dim"

#         self.backbone = nn.Sequential(
#             nn.Conv2d(1, 32, 4, stride=2, padding=0),
#             nn.ReLU(),
#             nn.Conv2d(32, 64, 4, stride=2, padding=0),
#             nn.ReLU(),
#             nn.Conv2d(64, 64, 3, stride=1, padding=0),
#             nn.ReLU(),
#             nn.AdaptiveAvgPool2d((res, res)),
#         )

#     def forward(self, x):
#         x = self.backbone(x).flatten(1)
#         x = BackboneOutput(encodings=x)
#         return x

In [ ]:
# #| export
# from mawm.models.misc import  (
#     build_mlp,
#     Projector,
#     MLP,
#     build_norm1d,
#     PartialAffineLayerNorm,
# )

# class MLPEncoder(SequenceBackbone):
#     def __init__(self, cfg, input_dim):
#         super().__init__()
#         self.encoder = MLP(
#             arch=cfg.backbone_subclass,
#             input_dim=input_dim,
#             norm=cfg.backbone_norm,
#         )

#     def forward(self, x):
#         x = self.encoder(x)
#         x = BackboneOutput(encodings=x)
#         return x


In [ ]:
# #| export
# class ObposEncoder1(SequenceBackbone):
#     """
#     Fused encoder for observation and pos state.
#     cat(obs, pos) --> encoder --> encodings
#     """

#     def __init__(self, config, obs_dim):
#         super().__init__()
#         self.config = config

#         self.encoder = MLP(
#             arch=config.backbone_subclass,
#             input_dim=obs_dim + config.pos_dim,
#             norm=config.backbone_norm,
#             activation="mish",
#         )

#         out_dim = int(config.backbone_subclass.split("-")[-1])

#         if config.final_ln:
#             self.final_ln = build_norm1d(config.backbone_norm, out_dim)
#         else:
#             self.final_ln = nn.Identity()

#     def forward(self, obs, pos):
#         x = torch.cat([obs, pos], dim=-1)
#         x = self.encoder(x)
#         x = self.final_ln(x)

#         return BackboneOutput(encodings=x)

In [ ]:
# #| export
# class ObposEncoder2(SequenceBackbone):
#     """
#     Distangled encoder for observation and pos state.
#     obs --> obs_encoder --> obs_out
#     pos --> pos_encoder --> pos_out
#     encodings = cat(obs_out, pos_out)
#     return: encodings, obs_out, pos_out
#     """

#     def __init__(self, config, obs_dim):
#         super().__init__()
#         self.config = config

#         obs_subclass, pos_subclass = config.backbone_subclass.split(",")

#         if obs_subclass == "id":
#             self.obs_encoder = nn.Identity()
#             obs_out_dim = obs_dim
#         else:
#             self.obs_encoder = build_mlp(
#                 layers_dims=obs_subclass,
#                 input_dim=obs_dim,
#                 norm=config.backbone_norm,
#                 activation="mish",
#             )
#             obs_out_dim = int(obs_subclass.split("-")[-1])

#         if pos_subclass == "id":
#             self.pos_encoder = nn.Identity()
#             pos_out_dim = config.pos_dim
#         else:
#             self.pos_encoder = build_mlp(
#                 layers_dims=pos_subclass,
#                 input_dim=config.pos_dim,
#                 norm=config.backbone_norm,
#                 activation="mish",
#             )
#             pos_out_dim = int(pos_subclass.split("-")[-1])

#         if config.final_ln:
#             self.final_ln = PartialAffineLayerNorm(
#                 first_dim=obs_out_dim,
#                 second_dim=pos_out_dim,
#                 first_affine=(obs_subclass != "id"),
#                 second_affine=(pos_subclass != "id"),
#             )
#         else:
#             self.final_ln = nn.Identity()

#     def forward(self, obs, pos):
#         obs_out = self.obs_encoder(obs)
#         pos_out = self.pos_encoder(pos)

#         next_state = torch.cat([obs_out, pos_out], dim=1)
#         next_state = self.final_ln(next_state)

#         return BackboneOutput(
#             encodings=next_state,
#             obs_component=obs_out,
#             pos_component=pos_out,
#         )



### Our Encoder

In [ ]:
#| export
import torch
import torch.nn as nn
from mawm.models.utils import Expander2D, build_conv

class GridMLP(nn.Module):
    def __init__(self, grid_width= 15, grid_height= 15, embed_dim= 8, w= 15, h= 15):
        super().__init__()
        
        self.x_embeddings = nn.Embedding(grid_width, embed_dim)
        self.y_embeddings = nn.Embedding(grid_height, embed_dim)
        
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim * 2, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            Expander2D(w, h)
        )
        
    def forward(self, x):
        """
        Input Shape:  [T, B, C, H, W]
        Output Shape: [T, B, C`, H`, W`]
        """

        if x.dim() == 3:
            x_coords, y_coords = x[:,:, 0], x[:,:, 1]
        else:
            x_coords, y_coords = x[:, 0], x[:, 1]
            
        x_vec = self.x_embeddings(x_coords)
        y_vec = self.y_embeddings(y_coords)
        
        combined = torch.cat([x_vec, y_vec], dim=-1)
        return self.mlp(combined)



In [ ]:
#| hide
T= 17
B= 32
model = GridMLP(grid_width=15, grid_height=15, embed_dim=8, w= 15, h= 15)
inp_pos = torch.randint(0, 15, (B, T, 2))  # Example x and y coordinates
model_output = model(inp_pos)
model_output.shape

torch.Size([32, 17, 16, 15, 15])

In [ ]:
#| export
import torch
import torch.nn as nn
from mawm.models.utils import Expander2D, build_conv

class MeNet6(nn.Module):
    def __init__(
        self,
        config,
        input_dim: int,
    ):
        super().__init__()

        self.config = config
        self.input_dim = input_dim
        subclass = config.backbone_subclass
        layers_config = ENCODER_LAYERS_CONFIG[subclass]

        if "l2" in subclass:
            # add prenormalization and relu layers?
            pre_conv = nn.Sequential(nn.GroupNorm(4, layers_config[0][0]), nn.ReLU())
        else:
            pre_conv = nn.Identity()
        conv_layers = build_conv(layers_config, (input_dim[0],))

        self.layers = nn.Sequential(pre_conv, conv_layers)

        if config.position_dim:
            # infer output dim of encoder
            sample_input = torch.randn(input_dim).unsqueeze(0) # [1, C, H, W]
            sample_output = self.layers(sample_input)
            encoder_output_dim = tuple(sample_output.shape[1:])

            if (
                self.config.position_encoder_arch
                and self.config.position_encoder_arch != "id"
            ):
                self.position_encoder = GridMLP(grid_width= 15, grid_height= 15, embed_dim= 8,
                                               w=encoder_output_dim[-2], h=encoder_output_dim[-1])
            else:
                self.position_encoder = Expander2D(
                    w=encoder_output_dim[-2], h=encoder_output_dim[-1]
                )
        
    @property
    def repr_dim(self):
        with torch.no_grad():
            sample_inp = torch.randn(self.input_dim).unsqueeze(0)  # [1, 1, C, H, W]
            sample_out = self.layers(sample_inp).unsqueeze(0)
            encoder_output_dim = tuple(sample_out.shape[1:])

            if self.config.position_dim:
                sample_pos = torch.randint(0, 15, (1, self.config.position_dim)).unsqueeze(0)
                sample_out_pos = self.position_encoder(sample_pos)
                pos_enc_output_dim = tuple(sample_out_pos.shape[1:])
                t = ([ax1 + ax2 for ax1, ax2 in zip(encoder_output_dim[1:2], pos_enc_output_dim[1:2])][0],
                     *encoder_output_dim[2:])
                return t
                
            return encoder_output_dim
    
    def forward(self, x, position=None):
        """
        input:
            x: [T, BS, *] or [BS, T, *]
        output:
            x: [T, BS, *] or [T, BS, *]
        """
        time= False
        if x.dim() == 2 or x.dim() == 4:
            encoded_obs = self.layers(x) # # [BS, C, H, W]
        else:
            time= True
            obs = x.flatten(0, 1) # [T*BS, C, H, W]
            encoded_obs = self.layers(obs) # [T*BS, 16, 10, 10]

        if position is not None:
            if position.dim() == 3:
                position = position.flatten(0, 1)  # [T*BS, pos_dim]
            encoded_pos = self.position_encoder(position)  # [T*BS, pos_dim, H, W]
            z = torch.cat([encoded_obs, encoded_pos], dim=1)
        else:
            encoded_pos = None
            z = encoded_obs
            
        if time:
            new_shape = x.shape[:2] + z.shape[1:]
            z = z.reshape(new_shape)
            return z
        return z
    

In [ ]:
#| hide
from omegaconf import OmegaConf


In [ ]:
#| hide
cfg = OmegaConf.load("../cfgs/MPCJepa/mpc.yaml")

cfg.model.backbone.position_dim = 2

In [ ]:
#| hide
model = MeNet6(
    config=cfg.model.backbone,
    input_dim=(3, cfg.model.img_size, cfg.model.img_size),
)

In [ ]:
#| hide
model.position_encoder

GridMLP(
  (x_embeddings): Embedding(15, 8)
  (y_embeddings): Embedding(15, 8)
  (mlp): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): Expander2D()
  )
)

In [ ]:
#| hide
T = 8
BS = 16
inp = torch.randn(BS, T, 3, 42, 42)
inp_pos = torch.randint(0, 15, (BS, T, 2))  # Example x and y coordinates


In [ ]:
#| hide
out = model(inp, position=inp_pos)
out.shape

torch.Size([16, 8, 32, 15, 15])

In [ ]:
## | hide
model.repr_dim

(1, 16, 15, 15) (1, 16, 15, 15)


(32, 15, 15)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()